## Valuation of Kiwibank using multiples ##

The code shows the valuation of Kiwibank on 22 August 2022, when the New Zealand government [announced its aquisition](https://www.beehive.govt.nz/release/kiwibank-remain-fully-kiwi-owned). 

I use a valuation method based on multiples, because traditional valuation approaches, based on discounting cashflows, do not always work well. The discount factor is low and volatile, which will result in large valuation swings.

The approach I take is well-documented and empirically tested. See, for example, this academic paper: [Equity Valuation Using Multiples Jing Liu, Doron Nissim, Jacob Thomas](https://onlinelibrary.wiley.com/doi/10.1111/1475-679X.00042).

The approach relies on a group of comparable firms. These firms are very similar to the firm one wants to value. The difference is that the comparable firms are all listed, so they all have a market value. For these firms, it is straightforward to find a representative P/E ratio (Price to Earnings) or Price to Book ratio. To determine the value of an unlisted firm, you multiply these ratios by the earnings or book value of the firm you are interested in.

To illustrate: suppose the average Price to Earnings ratio of the group of comparable firms is 15 and your (unlisted) firm reports a profit of 1 million dollars. In this case, investors in the group of comparable firms are willing to pay 15 dollars for a dollar of profit. With a 1 million dollar profit, the value of your firm is 15 million dollars. Likewise, using the Market to Book ratio: if the average M/B ratio is 1.25, and the value of book equity of your unlisted firm is 800 million dollars, its market value is 1 billion dollars. 

The challenging part of this approach is finding the right group of comparable banks. For Kiwibank, I chose banks from OECD countries plus banks from European countries that are not OECD members. I excluded banks from the US and Japan (because the accounting is different in the US, and because there are too many Japanese banks ending up in my sample). I also selected banks that are about the same size as Kiwibank and I filtered out banks with extreme values of critical variables, as  such profitability and capitalization. 

After determining the proper group of comparable banks, I determined the P/E ratio and the P/B (Price to Book) ratio and multiplied these ratios with Kiwibank's income number and book value.

<br>

I use Python to show you my workings. The data is from Datastream and the variables I use are shown below. You should be able to replicate my work.

Initiate Python, variables, countries

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
variables = {'X(MV)~E': 'MV', 'PE': 'PE', 'X(WC05476)~E': 'BVPS', 'MTBV': 'MTBV', 'PTBV': 'PTBV', 'X(DWTA)~E': 'DWTA', 'X(EPS)~E': 'EPS', 'X(P)~E': 'P', 'NOSH': 'NOSH', 'X(WC01751)~E': 'NI', 'NAME': 'NAME', 'country_name': 'country', 'X(WC18228)~E': 'Tier1', 'WC18157': 'Tier1Ratio', 'X(WC18156)~E': 'RWA', 'date': 'date'} # All values retrieved in Euros
country_list = pd.read_csv('oecd_eu_oz_nz.csv').set_index('country')

Retrieve data from a Datastream Request table

In [2]:
df = pd.read_excel("DFORequestTable2.xlsm", sheet_name="values").set_index('GGISO').join(country_list).rename(columns=variables).set_index('Type').dropna(subset=['country', 'EPS', 'DWTA', 'MTBV', 'NI', 'PE'])
df = df.assign(dual=df.NAME.str.contains('\([A-Z]+\)', regex=True, na=False),
               NOSH=df.NOSH.replace(0, np.NaN))

keep = ['NAME', 'country', 'NOSH', 'PE', 'MTBV', 'PTBV', 'NI', 'RWA', 'Tier1Ratio', 'MV', 'DWTA', 'EPS', 'P', 'BVPS', 'Tier1', 'date']

Determine additional variables and make some adjustments.

In [3]:
def bv_calc(df):
    return df.NOSH.mul(df.BVPS)


df = df[keep].assign(BV=bv_calc(df),
               MV=df.MV.mul(1000),  # mv in millions  bv in thousands
               leverage=bv_calc(df).div(df['DWTA']),
               roe=df['EPS'].div(df['BVPS']).replace(0, np.NaN),
               roa=df['NI'].div(df['DWTA']).replace(0, np.NaN),
               density=df['RWA'].div(df['DWTA']),
               Tier1Ratio=df['Tier1Ratio'].div(100)
               ).drop('TK:TBB').loc[df.dual == False]

df.sample(5)

,NAME,country,NOSH,PE,MTBV,PTBV,NI,RWA,Tier1Ratio,MV,...,EPS,P,BVPS,Tier1,date,BV,leverage,roe,roa,density
Type,,,,,,,,,,,,,,,,,,,,,
DK:LOL,LOLLANDS BANK,Denmark,1080.0,4.2,1.01,1.01,9898.0,NaN,NaN,83350.0,...,18.45,77.18,85.541,NaN,2021-12-31,9.238428e+04,0.156172,0.215686,0.016732,NaN
N:DNB,DNB BANK,Norway,1550364.0,13.3,1.38,1.38,2433851.0,97069941.0,0.210,31229490.0,...,1.52,20.14,14.580,20382642.0,2021-12-31,2.260431e+07,0.077667,0.104252,0.008363,0.333527
S:BCVN,BC VAUD N,Switzerland,86062.0,17.6,1.67,1.67,365487.0,18592861.0,0.172,5880600.0,...,3.89,68.33,40.956,3192587.0,2021-12-31,3.524755e+06,0.065273,0.094980,0.006768,0.344313
O:KAER,BKS BANK,Austria,42943.0,6.3,0.47,0.46,80759.0,5943800.0,0.130,657030.0,...,2.45,15.30,33.075,774700.0,2021-12-31,1.420340e+06,0.134381,0.074074,0.007641,0.562355
DK:RIL,RINGKJOBING LANDBOBANK,Denmark,29068.0,22.1,2.93,2.86,165278.0,5819800.0,0.176,3431430.0,...,5.33,118.05,41.249,1026108.0,2021-12-31,1.199026e+06,0.147778,0.129215,0.020370,0.717280


We now have 255 banks:

In [4]:
df.shape

(255, 21)

Convert currencies to NZD and re-order the dataframe columns.

In [5]:
vals = ['MV', 'DWTA', 'EPS', 'P', 'BVPS', 'Tier1']
ids = ['NAME', 'country', 'NOSH']
dfc = df[ids].join(df[vals].div(0.6016).div(1000)).join(df[[x for x in df if x not in vals and x not in ids]])
df.sample(5)

,NAME,country,NOSH,PE,MTBV,PTBV,NI,RWA,Tier1Ratio,MV,...,EPS,P,BVPS,Tier1,date,BV,leverage,roe,roa,density
Type,,,,,,,,,,,,,,,,,,,,,
RM:BRD,BRD - GROUPE SG,Romania,696901.0,10.8,1.26,1.26,264649.0,6337332.0,0.2077,2492570.0,...,0.3307,3.5766,2.848,1316085.0,2021-12-31,1984774.048,0.142592,0.116117,0.019013,0.455292
CR:HRV,HRVATSKA POSTANSKA BANKA,Croatia,2025.0,6.6,0.56,0.56,26954.0,NaN,NaN,196600.0,...,14.7300,97.1100,174.623,NaN,2021-12-31,353611.575,0.095501,0.084353,0.007280,NaN
O:KAER,BKS BANK,Austria,42943.0,6.3,0.47,0.46,80759.0,5943800.0,0.1300,657030.0,...,2.4500,15.3000,33.075,774700.0,2021-12-31,1420339.725,0.134381,0.074074,0.007641,0.562355
S:SGKN,ST GALLER KANTONALBANK,Switzerland,5994.0,14.7,0.98,0.97,174922.0,15538032.0,0.1600,2516280.0,...,28.6200,419.8230,431.252,2486085.0,2021-12-31,2584924.488,0.067668,0.066365,0.004579,0.406754
PO:MBK,MBANK,Poland,42385.0,66.1,1.34,1.34,-257182.0,20886115.0,0.1416,4006060.0,...,1.4300,94.5200,70.605,2956805.0,2021-12-31,2992592.925,0.069222,0.020254,-0.005949,0.483117


Now select comparable banks that are similar to Kiwibank in size, leverage, performace, etc

In [6]:
dfc = dfc[dfc['DWTA'].between(10000,  150000)]  # Total assets
dfc = dfc[dfc['leverage'].between(0.02, 0.20)]
dfc = dfc[dfc['roe'].between(0.02,  0.25)]  # Exclude extreme performers
dfc = dfc[dfc['Tier1Ratio'].between(0.06,  0.25)]  # Include banks that meet capital requirements and have no extreme Tier 1 ratios
dfc = dfc[dfc['MTBV'].between(0.25,  5)]  # Exclude extreme Market to Book ratios
dfc = dfc[dfc['PE'].between(2, 20)] # Exclude extreme P/E ratios

dfc.sample(5)

,NAME,country,NOSH,MV,DWTA,EPS,P,BVPS,Tier1,PE,...,PTBV,NI,RWA,Tier1Ratio,date,BV,leverage,roe,roa,density
Type,,,,,,,,,,,,,,,,,,,,,
N:NONG,SPAREBANK 1 NORD-NORGE,Norway,100398.0,1873.853059,19754.913564,0.001463,0.018667,0.011998,2320.762965,12.8,...,1.56,105543.0,6986240.0,0.2000,2021-12-31,724672.764,0.060976,0.121917,0.008881,0.587842
IS:FIBI,FIRST INTL.BK.OF ISR.,Israel,100330.0,6101.845080,84465.242686,0.006001,0.060818,0.046823,4789.805519,10.1,...,1.30,396958.0,25141988.0,0.1146,2021-12-31,2826195.770,0.055618,0.128155,0.007812,0.494782
TK:YKB,YAPI VE KREDI BANKASI,Turkey,8447050.0,3142.603059,85524.092420,0.000100,0.000366,0.000828,7410.839428,3.7,...,0.45,694609.0,29957576.0,0.1488,2021-12-31,4206630.900,0.081759,0.120482,0.013500,0.582251
D:PCZ,PROCREDIT HOLDING,Germany,58898.0,783.228059,13653.848072,0.001961,0.013298,0.024167,1316.472739,6.8,...,0.55,79642.0,5600891.0,0.1410,2021-12-31,856318.022,0.104249,0.081161,0.009696,0.681858
SV:NVB,NOVA LJUBLJANSKA BANKA,Slovenia,20000.0,2533.244681,35802.059508,0.030801,0.126662,0.172768,3267.205785,4.1,...,0.73,236404.0,12667408.0,0.1550,2021-12-31,2078740.000,0.096513,0.178281,0.010976,0.588128


We now have 71 banks

In [7]:
dfc.shape

(71, 21)

Now use the data from the group of comparables to determine the value of Kiwibank.

Note that the value of the group of comparable firms increased by about 4 percent from the start of the year up to 22 August.

In [8]:
def valuation(df, nosh, driver, var, mv_appreciation):
    print(f'Value increase since start of the year: {mv_appreciation*100.0:4.3f} percent.')
    if var in ['MTBV', 'PTBV', 'PE']:
        k = stats.hmean(df[var], axis=0)  # According to Liu et al 2002, use the Harmonic mean, not the average
        if var == 'PE':
            print(f'Earnings Multiple: {k:4.2f}')
            print(f'EPS: ${driver / nosh:4.3f}.')
        elif var in ['MTBV', 'PTBV']:
            print(f'Book Multiple: {k:4.3f}')
            print(f'Book value per share: ${driver / nosh:4.3f}.')
        else:
            return
        print(f'Price per share: ${driver/nosh*k:4.5f}')
        valuation = driver * k / 1e6 * (1 + mv_appreciation)
        print(f'Valuation: ${valuation:4,.2f} in millions.')
    else:
        valuation = 'error'
    print('\n')
    return valuation


Use these inputs for Kiwibank

In [9]:
# From RBNZ Dashboard
profit_jun_22 = 131_300_000
book_value_jun_22 = 2_203_500_000

# Number of kiwibank shares
kiwi_nosh = 1_458_157_403
overall_delta_mv= 0.04_833_342

Determine the value

In [10]:
v1 = valuation(dfc, kiwi_nosh, profit_jun_22,     'PE',   overall_delta_mv)
v2 = valuation(dfc, kiwi_nosh, book_value_jun_22, 'PTBV', overall_delta_mv)  # using Price to Book ratios
v3 = valuation(dfc, kiwi_nosh, book_value_jun_22, 'MTBV', overall_delta_mv)  # using Market to Book ratios

print(f'Average Valuation: ${(v1+v2+v3)/3:4,.2f} in millions.')

Value increase since start of the year: 4.833 percent.
Earnings Multiple: 9.24
EPS: $0.090.
Price per share: $0.83224
Valuation: $1,272.19 in millions.


Value increase since start of the year: 4.833 percent.
Book Multiple: 0.778
Book value per share: $1.511.
Price per share: $1.17575
Valuation: $1,797.30 in millions.


Value increase since start of the year: 4.833 percent.
Book Multiple: 0.783
Book value per share: $1.511.
Price per share: $1.18382
Valuation: $1,809.63 in millions.


Average Valuation: $1,626.37 in millions.
